In [1]:
import csv, sys, ast

from pathlib import Path
from tqdm import tqdm

import dask.dataframe as dd
import pandas as pd
import numpy as np

in_file = Path('data/temp_train_news_data.csv') # to be deleted
train_file = Path('data/train_news_data.csv')
valid_file = Path('data/valid_news_data.csv')

In [2]:
# https://stackoverflow.com/questions/15063936/csv-error-field-larger-than-field-limit-131072

maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [3]:
chunksize = 100000
reader = pd.read_csv(in_file, chunksize=chunksize, encoding='utf-8', engine='python')

for i, chunk in enumerate(tqdm(reader)):
    # split into training and valdiation dataset 80-20
    # and shuffle data
    train, valid = np.split(chunk.sample(frac=1).reset_index(drop=True), [int(.8 * len(chunk))])
    
    # drop NaN
    train = train.dropna()
    valid = valid.dropna()
    
    if not valid_file.exists():
        train.to_csv(train_file, index=False)
        valid.to_csv(valid_file, index=False)
    else:
        train.to_csv(train_file, index=False, mode='a', header=False)
        valid.to_csv(valid_file, index=False, mode='a', header=False)

86it [14:56, 10.82s/it]


Check if csv files have actually no null values

In [4]:
df = dd.read_csv(train_file)
df.isnull().sum().sum().compute()

0

In [5]:
df = dd.read_csv(valid_file)
df.isnull().sum().sum().compute()

0